# Hidden Markov Models

## Manuelles Berechnen eines HMMs
Für eine Smartwatch soll ein System entwickelt werden, das automatisch erkennt, ob der Träger gerade schläft. Die Smartwatch enthält einen Beschleunigungssensor, der entweder den Wert "wenig Bewegung" oder den Wert "viel Bewegung" zurückliefert. Wenn der Träger schläft, ist die Wahrscheinlichkeit für "viel Bewegung" 5 % und die Wahrscheinlichkeit für "wenig Bewegung" 95 %.
Wenn der Träger wach ist, ist die Wahrscheinlichkeit für "viel Bewegung" 60 % und die Wahrscheinlichkeit für "wenig Bewegung" 40 %. 

Wenn der Träger schläft, ist die Wahrscheinlichkeit, dass er im nächsten Zeitschritt wach ist, 30 %. Wenn der Träger wach ist, ist die Wahrscheinlichkeit, dass er im nächsten Schritt schläft, 20 %. Zu Anfang schläft der Träger mit einer Wahrscheinlichkeit von 50 %.

1. Spezifizieren Sie diesen Prozess als HMM, wobei der Zustand beschreibt, ob die Person schläft oder wach ist. Notieren Sie die Initialzustand und das Beobachtungsmodell.

2. Berechnen Sie, ausgehend von der initialen Wahrscheinlichkeitsverteilung über den Zuständen, ...

    a) die Vorhersage, d.h., Wahrscheinlichkeitsverteilung allein auf Basis des Transitionsmodelles!
    
    b) die Korrektur nach der Beobachtung "wenig Bewegung"!

In [1]:
#transition model
T = matrix(c(0.8,0.2,0.3,0.7),nrow=2,ncol=2)

#prior
s0 = c(0.5,0.5)

#obs model
O = matrix(c(0.6,0.4,0.05,0.95),nrow=2,ncol=2)

#solve the manual computation exercise:
s1 = T %*% s0
s1i = O[2,] * s1 #2nd row of matrix because y="wenig bewegung"
s2 = s1i / sum(s1i)

In [9]:
s2

0.3397683
0.6602317


## HMM Implementierung

Implementieren Sie das oben beschriebene Hidden Markov Model in R. Die Verteilung $p(x_t | y_{1:t})$ soll durch einen Vektor $s$ dargestellt werden, wobei das erste Element des Vektors die Wahrscheinlichkeit $p(x_t = Wach)$ und das zweite Element die Wahrscheinlichkeit $p(x_t = Schlafend)$ angibt. Das Transitionsmodell $p(x_{t+1} | x_{t})$ soll als Matrix $T$ dargestellt werden, wobei der Eintrag $t_{ij}$ die Wahrscheinlichkeit $p(x_{t+1} = i | x_{t} = j)$ darstellt. Das Observationsmodell $p(y_t | x_t)$ soll ebenfalls als Matrix dargestellt werden, wobei der Eintrag $o_{ij}$ die Wahrscheinlichkeit $p(y_t=i | x_t=j)$ angibt.

Folgende Funktionen sollen implementiert werden:

1. `predict(s,T)`: Gegeben eine Prior-Verteilung $p(x_t | y_{1:t})$ als Vektor $s$ und ein Transitionsmodell, berechne die Verteilung nach dem Prädiktionsschritt, d.h. $p(x_{t+1} | y_{1:t})$.
2. `update(s,O,y)`: Gegeben eine Verteilung $p(x_{t+1} | y_{1:t})$ als Vektor $s$, ein Observationsmodell $O$ und eine Beobachtung $y_{t+1}$, berechne die Verteilung nach dem Korrektur-Schritt, d.h. $p(x_{t+1} | y_{1:t+1})$.
3. `filter(s0,Y,T,O)`: Gegeben eine Prior-Verteilung $p(x_0)$ als Vektor s0, eine Sequenz (d.h. ein Vektor) von Beobachtungen $y_{1:T}$, ein Transitionsmodell T und Beobachtungsmodell O, berechne für jedes $t = 1,...,T$ die Verteilung $p(x_t | y_{1:t})$.

In [10]:
predict = function(s,T){
  (T %*% s)[,1]
}

update = function(s,O,y){
  orow = ifelse(y=="V",1,2)
  si = O[orow,] * s
  si / sum(si)
}

filter = function(s0,O,T,yy){
  s = s0
  ss = list(s)
  #for each y...
  for(y in yy){
    si = predict(s,T)
    s = update(si,O,y)
    ss = c(ss,list(s))
  }
  ss
}

testHMM = function(){
  #transition model
  T = matrix(c(0.8,0.2,0.3,0.7),nrow=2,ncol=2)
  #prior
  s0 = c(0.5,0.5)
  #obs model
  O = matrix(c(0.6,0.4,0.05,0.95),nrow=2,ncol=2)
  #observation sequence
  yy = c("W","W","V")
  ss = filter(s0,O,T,yy)
}


In [13]:
ss = testHMM()
ss

[[1]]
[1] 0.5 0.5

[[2]]
[1] 0.3397683 0.6602317

[[3]]
[1] 0.2717807 0.7282193

[[4]]
[1] 0.90265233 0.09734767